# WEATHER CHAT BOT

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from itertools import chain
from nltk import NaiveBayesClassifier as nbc
import codecs
import json
from urllib.request import urlopen
import datefinder
import datetime,time
from dateutil import parser
from nltk.tag.stanford import StanfordNERTagger
import pywapi
import random
import string

# Get Default Location

In [2]:
def get_default_location():

    reader = codecs.getreader("utf-8")
    url = 'http://ipinfo.io/json'
    response = urlopen(url)
    data = json.load(reader(response))
    city = data['city']
    region=data['region']
    if_location_not_given = city + ", " + region
    return if_location_not_given

# Get date

In [3]:
def get_day_of_week(question):

    
    question = question.lower()
    matches = list(datefinder.find_dates(question))

    if len(matches) > 0:
        date = matches[0]
        day = date.strftime('%A')
    else:
        if 'today' in question:
            day = ((datetime.date.today()).strftime('%A'))
        elif 'day after tomorrow' in question:
            day = ((datetime.date.today() + datetime.timedelta(days = 2)).strftime('%A'))
        elif 'tomorrow' in question:
            day = ((datetime.date.today() + datetime.timedelta(days = 1)).strftime('%A'))
        else:
            day = ((datetime.date.today()).strftime('%A'))
    return day

# Get Location

In [4]:
def get_location(question):
    
    location_tagger = StanfordNERTagger('Tagger/stanford-ner-2016-10-31/classifiers/english.all.3class.distsim.crf.ser.gz', 'Tagger/stanford-ner-2016-10-31/stanford-ner-3.7.0.jar')
    question = question.title()
    tag = location_tagger.tag(question.split())
    loc_word = ''
    for word,tag in tag:
        if(tag == 'LOCATION'):
            loc_word = loc_word + " " + word
        loc_word = loc_word.strip()
    if loc_word == '':
        loc_word = get_default_location()
    return loc_word

# Get weather of that Day

In [5]:
def get_weather(city, day_of_week):
   

 # Get Weather.com Data
    count = 0
    lookup = pywapi.get_location_ids(city)
    while len(lookup) != 1:
        if len(lookup) > 1:
            print('Choose the correct one: ')
            for i,j in zip(lookup.values(),range(1,len(lookup.values())+1)):
                print(j,' Which one do you want ?',i)
            city = input('Enter Location: ')
            lookup = pywapi.get_location_ids(city)
        if len(lookup) == 0:
            print('Curent Location ',city,' is unable to locate')
            city = input('Please provide the location correctly: ')
            count += 1
            lookup = pywapi.get_location_ids(city)
            if count == 2:
                city = input('Please provide the best nearest location, we are unable to find that for you : ')
                lookup = pywapi.get_location_ids(city)
    for k in lookup:
        location_id = k
    
# Extracting the weather forcast for that location 

    weather_com = pywapi.get_weather_from_weather_com(location_id)
    now_temp = weather_com['current_conditions']['temperature']
    weather_forecast = pd.DataFrame.from_dict(weather_com['forecasts'])
    weather_update = weather_com['current_conditions']['last_updated']
    weather_forecast_night_data = pd.DataFrame.from_dict(weather_forecast['night'])
    weather_forecast_day_data = pd.DataFrame.from_dict(weather_forecast['day'])
    weather_forecast_day_night_data = weather_forecast_day_data.join(weather_forecast_night_data)

#Extracting just relevant data

    main_list = []
    time_list = []
    night_day_data = {}
    for time in weather_forecast_day_night_data:
        for i in range(len(weather_forecast_day_night_data[time])):
            for key,value in weather_forecast_day_night_data[time][i].items():
                if key == 'chance_precip':
                    n_key = time + '_chances_of_rain'
                    night_day_data[n_key] = value + '%'
                if key == 'text':
                    n_key = time + '_weather'
                    if value == '':
                        value = 'Clear'
                    night_day_data[n_key] = value
                if key == 'humidity':
                    n_key = time + '_humidity'
                    night_day_data[n_key] = value
            time_list.append(night_day_data)
            night_day_data = {}
        main_list.append(time_list)
        time_list = []

# Merging the needful Data

    day_data = pd.DataFrame.from_dict(main_list[0])
    night_data = pd.DataFrame.from_dict(main_list[1])
    day_data = day_data.join(night_data)
    weather_forecast['current temperature'] = now_temp
    high_data = pd.DataFrame(weather_forecast['high']).join(pd.DataFrame(weather_forecast['low']))
    high_data_day = pd.DataFrame(weather_forecast['day_of_week']).join(high_data)
    weather = high_data_day.join(day_data)
    weather = weather.join(pd.DataFrame(weather_forecast['current temperature']))
# Getting for that day

    weather = weather.set_index(['day_of_week'])
    if day_of_week in weather.index.tolist():
        final_data = pd.DataFrame(weather.ix[day_of_week])
        return final_data,weather_update,city
    else:
        return None, None, None

# Getting the train Data

In [98]:
def get_traindata():
    train_csv = pd.read_csv('weathertrain.csv',header= None,names = ['sentence','label'])
    subset = train_csv[['sentence', 'label']]
    tuples = [tuple(x) for x in subset.values]
    return tuples

                                           sentence        label
0                                      is it rainy?         rain
1                               will it rain today?         rain
2                         how will the weather be ?      weather
3                              how is the weather ?      weather
4                       how’s gonna be the weather?      weather
5                                     is it cloudy?      weather
6                          what’s the weather like?      weather
7                                     is it sunny ?      weather
8                              is it rainy outside?         rain
9                      how is today’s weather like?      weather
10                                will it drizzle ?         rain
11                             is the weather fine?      weather
12                                    will it rain?         rain
13                           will it rain tomorrow?         rain
14                 how is

[('is it rainy?', 'rain'),
 ('will it rain today?', 'rain'),
 ('how will the weather be ?', 'weather'),
 ('how is the weather ?', 'weather'),
 ('how’s gonna be the weather?', 'weather'),
 ('is it cloudy?', 'weather'),
 ('what’s the weather like?', 'weather'),
 ('is it sunny ?', 'weather'),
 ('is it rainy outside?', 'rain'),
 ('how is today’s weather like?', 'weather'),
 ('will it drizzle ?', 'rain'),
 ('is the weather fine?', 'weather'),
 ('will it rain?', 'rain'),
 ('will it rain tomorrow?', 'rain'),
 ('how is the weather in bengaluru?', 'weather'),
 ('any chance of rain today?', 'rain'),
 ('what is the temperature ', 'temperature'),
 ('is it raining in chennai?', 'rain'),
 ('how is the weather in Pune?', 'weather'),
 ('will it rain in pune?', 'rain'),
 ('get me today’s weather', 'weather'),
 ('what is today’s min temperature ', 'temperature'),
 ('what is today’s minimum temperature', 'temperature'),
 ('what is the temperature ', 'temperature'),
 ('can i expect any showers?', 'rain'),

# Training the data

In [7]:
def trainNBC():
    data = get_traindata()
    stop_words = set(stopwords.words("english"))
    d1 = [(' '.join(list((i for i in word_tokenize(sentence) if i not in stop_words))),tag) for sentence, tag in data ]
    vocabulary = set(chain(*[word_tokenize(i[0].lower()) for i in d1 if i not in stop_words]))
    feature_set = [({i:(i in word_tokenize(sentence.lower())) for i in vocabulary},tag) for sentence, tag in d1]
    classifier = nbc.train(feature_set)
    return vocabulary,classifier

# Classifier for getting class

In [8]:
def get_class(query):
    test_sentence = query
    vocabulary,classifier = trainNBC()
    featurized_test_sentence =  {i:(i in word_tokenize(test_sentence.lower())) for i in vocabulary}
    ans = classifier.classify(featurized_test_sentence)
    return ans

# Method to get Class Relevant Data

In [92]:
def get_output(query):
    temperature = {}
    weather_text = {}
    rain_data = {}
    full_details = {}
    temperature['Highest temperature'] = weather.ix['high'][day_of_week]
    temperature['Lowest temperature'] = weather.ix['low'][day_of_week]
    temperature['Current temperature'] = weather.ix['current temperature'][day_of_week]
    weather_text['During Day'] = weather.ix['day_weather'][day_of_week]
    weather_text['During Night'] = weather.ix['night_weather'][day_of_week]
    rain_data['During Day'] = weather.ix['day_chances_of_rain'][day_of_week]
    rain_data['During Night'] = weather.ix['night_chances_of_rain'][day_of_week]
    full_details['Day Time'] = [weather.ix['day_weather'][day_of_week],weather.ix['day_chances_of_rain'][day_of_week],weather.ix['day_humidity'][day_of_week]]
    full_details['Night Time'] = [weather.ix['night_weather'][day_of_week],weather.ix['night_chances_of_rain'][day_of_week],weather.ix['night_humidity'][day_of_week]]
    temp_pd = pd.DataFrame.from_dict(temperature,orient='index')
    rain_pd = pd.DataFrame.from_dict(rain_data,orient='index')
    weather_pd = pd.DataFrame.from_dict(weather_text,orient='index')
    full_details_pd = pd.DataFrame.from_dict(full_details,orient='index')
    temp_pd.columns = ['temperature']
    temp = pd.DataFrame(temp_pd)
    rain_pd.columns = ['rain']
    weather_pd.columns = ['weather']
    full_details_pd.columns = ['Weather','Chances of Rain','Humidity']
    temp_pd = temp_pd.unstack(level=1)
    rain_pd = rain_pd.unstack(level=1)
    weather_pd = weather_pd.unstack(level=1)
    frames = [temp_pd, rain_pd, weather_pd]
    result = pd.DataFrame(pd.concat(frames))
    result.columns = ['Details'] 
    category = get_class(query)
    print(temp)
    if category == 'max':
        output = temp.ix['Highest temperature']['temperature']
    elif category == 'min':
        output = temp.ix['Lowest temperature']['temperature']
    elif category == 'temperature':
        output = temp.ix['Current temperature']['temperature']
    else:
        output = result.ix[category]
    return output

# Check for greetings 


In [93]:
GREETING_KEYWORDS = ("hello", "hi", "greetings", "sup", "what's up",)

GREETING_RESPONSES = ["'sup bro", "hey", "*nods*", "hey you get my snap?"]

def check_for_greeting(sentence):
    """If any of the words in the user's input was a greeting, return a greeting response"""
    for word in word_tokenize(sentence):
        if word.lower() in GREETING_KEYWORDS:
            return random.choice(GREETING_RESPONSES)
        else:
            return "Nothing"

# Get User Input and produce output

In [96]:
query = input('Hello! What do you want to know ? ')
greet = check_for_greeting(query)
while greet != 'Nothing':
    new_query = greet + ' How May I assist you ?'
    query = input(new_query)
    greet = check_for_greeting(query)
question = "".join(l for l in query if l not in string.punctuation)
city = get_location(question)
day_of_week = get_day_of_week(question)
weather, time, city = get_weather(city,day_of_week)
if city == None:
    print('Cannnot predict for data more than 5 days')
else:   
    output = get_output(query)
    #print("**** ",output.columns.tolist()[0].capitalize()," Details for ",city," on ",day_of_week," as updated on ",time," ****")
    weather_data = output
weather_data

Hello! What do you want to know ? wHAT IS THE coolest TEMPERATURE TODAY IN BENGALURU?
                    temperature
Current temperature          35
Lowest temperature           21
Highest temperature          33


'35'